In [2]:
import os
import codecs
import re
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from gensim.test.utils import common_texts

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
stopwords = ["баш","без","биће","био","бити","близу","број","дана","данас","доћи","добар","добити","док","доле","дошао","други","дуж","два","често","чији","где","горе","хвала","и","ићи","иако","иде","има","имам","имао","испод","између","изнад","изван","изволи","један","једини","једном","јесте","још","јуче","кад","како","као","кога","која","које","који","кроз","мали","мањи","мисли","много","моћи","могу","мора","морао","на","наћи","наш","негде","него","некад","неки","немам","нешто","није","ниједан","никада","нисмо","ништа","њега","његов","њен","њих","њихов","око","около","она","онај","они","оно","осим","остали","отишао","овако","овамо","овде","ове","ово","о","питати","почетак","поједини","после","поводом","правити","пре","преко","према","први","пут","радије","сада","смети","шта","ствар","стварно","сутра","сваки","све","свим","свугде","тачно","тада","тај","такође","тамо","тим", "у", "учинио","учинити","умало","унутра","употребити","узети","ваш","већина","веома","видео","више","захвалити","зашто","због","желео","жели","знати"]

In [4]:
#not using this one
dataset_path = '../webapp/public/acts'
dataset = {}
TAG_RE = re.compile(r'<[^>]+>')
for file_path in os.listdir(dataset_path):
    with codecs.open(os.path.join(dataset_path, file_path), 'r', encoding='utf-8') as f:
        data = f.read()
        data = data.lower() #lowercase
        data = TAG_RE.sub('', data) #remove html tags
        data = data.translate(str.maketrans('', '', string.punctuation + "„”")) #remove punctuation
        data_split = data.split() #remove extra whitespaces
        data_split = [word for word in data_split if word not in stopwords and not word.isnumeric()]
#         data = " ".join(data_split)
        dataset[file_path] = data_split

KeyboardInterrupt: 

In [5]:
#new
def hasNumbers(inputString):
     return any(char.isdigit() for char in inputString)

In [6]:
#new
clan_re = re.compile("<p>[ \t\n]*Члан[ \t\n]*[0-9]*\..*?<\/p>")
clan_re.split("sadsadasd<p>Члан  13.  </p>asdj<p>Члан  12.  </p>sadsad")

['sadsadasd', 'asdj', 'sadsad']

In [7]:
#using this one to load the dataset instead
start = time.time()
dataset_path = '../acts'
dataset = {}
TAG_RE = re.compile(r'<[^>]+>')
corpus_clans = []
for file_path in os.listdir(dataset_path):
    with codecs.open(os.path.join(dataset_path, file_path), 'r', encoding='utf-8') as f:
        data = f.read()
        clans = clan_re.split(data)
        clans_no_html = [TAG_RE.sub('', clan) for clan in clans] #remove html tags
        clans_no_white_spaces = [' '.join(clan.split()) for clan in clans_no_html]
        corpus_clans.append(clans_no_white_spaces)
        clans_no_punc = (clan.translate(str.maketrans('', '', string.punctuation + "„”–vi")) for clan in clans_no_white_spaces) #remove punctuation
        clans_lower = [clan.lower() for clan in clans_no_punc] #lowercase
        clans_no_stopwords = []
        for i, clan in enumerate(clans_lower):
            new_clan = clan.split()
            new_clan = [word for word in new_clan if word not in stopwords and not hasNumbers(word)]
            new_clan = ' '.join(new_clan)
            clans_no_stopwords.append(new_clan)
#         data = [row_clean for row in data if (row_clean := " ".join(row.split())) != ''] #remove extra whitespaces
        clans_clean = [" ".join(clan.split()) for clan in clans_no_stopwords] #remove extra whitespaces
        clans_cut = [clan[:500000] for clan in clans_clean]
        dataset[file_path] = clans_cut
end = time.time()
corpus = list(dataset.values())
print(end-start)

27.981695890426636


In [8]:
#new
sentences = [clan.split(" ") for zakon in corpus for clan in zakon]

In [9]:
#changed
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]
model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, workers=4)

In [77]:
model.infer_vector(["закон", "порез"], alpha=0.025, min_alpha=0.025, steps=50)[0]

-0.04978092

In [78]:
#moved up, it was 2 cells before
def query_to_vec(query, model):
    query = query.lower() #lowercase
    query = TAG_RE.sub('', query) #remove html tags
    query = query.translate(str.maketrans('', '', string.punctuation + "„”")) #remove punctuation
    query_split = query.split() #remove extra whitespaces
    query_split = [word for word in query_split if word not in stopwords and not word.isnumeric()]
    docvec = model.infer_vector(query_split, alpha=0.025, min_alpha=0.025, steps=50)
    return docvec

In [79]:
#changed
def dataset_to_vectors(dataset, model):
    """Returns dataset vectorized. Doesn't change the dataset"""
    X = []
    for key, clans in dataset.items():
        try:
            clans_embedding = [query_to_vec(clan, model) for clan in clans]
        except ValueError:
            clans_embedding = [[]]
        X.append(clans_embedding)
    return X

In [80]:
#new
embeddings = dataset_to_vectors(dataset, model)

In [94]:
import torch
with open('../saved_embeddings/doc2vec_embeddings.dv', 'wb') as f:
    torch.save(embeddings, f)

In [101]:
#new
with open('../saved_embeddings/doc2vec_embeddings.dv', 'rb') as f:
    embeddings2 = torch.load(f)

In [111]:
embeddings[0][2] == embeddings2[0][2]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [47]:
#not using this one
def calculate_similarity_word2vec(X, model, query, top_k=10):
    """ Vectorizes the `query` via `query_to_vec` and calculates the cosine similarity of
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""

    # Vectorize the query to the same length as documents
    query_vec = query_to_vec(query, model)
    query_vec = query_vec.reshape(1,-1)
    # Compute the cosine similarity between query_vec and all the documents
    cosine_similarities = cosine_similarity(X,query_vec).flatten()
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(cosine_similarities, axis=0)[:-top_k-1:-1]
    return (most_similar_doc_indices, cosine_similarities)

In [81]:
#new
def calculate_similarity(embeddings, model, query, top_documents=10, top_sentences=3):
    """ Embeds the `query` via `model` and calculates the cosine similarity of
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""

    # Vectorize the query to the same length as sentences in documents
    query_vec = query_to_vec(query, model)
    query_vec = query_vec.reshape(1,-1)
    # Compute the cosine similarity between query_vec and all the documents
    scores = np.array([])
    top_sentence_indices = []
    for i, zakon_embedding in enumerate(embeddings):
        if len(zakon_embedding[0]) > 0:
            cosine_scores = cosine_similarity(query_vec, zakon_embedding)[0]
        else:
            cosine_scores = np.array([0.])
        indices_best = cosine_scores.argsort()[-1:-top_sentences-1:-1]
        scores = np.append(scores, cosine_scores[indices_best].mean())
        top_sentence_indices.append(indices_best)
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(scores, axis=0)[-1:-top_documents-1:-1].astype(int)
    top_sentence_indices = np.array(top_sentence_indices)[most_similar_doc_indices]
    return (most_similar_doc_indices, scores, top_sentence_indices)

In [312]:
#not using this one
def show_similar_documents(df, cosine_similarities, similar_doc_indices):
    """ Prints the most similar documents using indices in the `similar_doc_indices` vector."""
    counter = 1
    for index in similar_doc_indices:
        print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
        print('body: {}, '.format(df[index]))
        print()
        counter += 1

In [82]:
#new 
def show_similar_documents(doc_titles, cosine_similarities, similar_doc_indices, corpus, similar_doc_sentence_indices):
    """ Prints the most similar documents using indices in the `similar_doc_indices` vector."""
    counter = 1
    
    for i, index in enumerate(similar_doc_indices):
        print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
        print('body: {}, '.format(doc_titles[index]))
        print('Most similar articles: ')
        for jandex in similar_doc_sentence_indices[i]:
            print(f'\t{corpus[index][jandex]}')
        print()
        counter += 1

In [115]:
#changed
top_documents = 10
top_sentences = 3
user_question = 'порез основа казна'
search_start = time.time()
sim_vecs, cosine_similarities, top_sentence_indices = calculate_similarity(embeddings, 
                                                                           model, 
                                                                           user_question, 
                                                                           top_documents=top_documents,
                                                                           top_sentences=top_sentences)
search_time = time.time() - search_start
print("search time: {:.2f} ms".format(search_time * 1000))
print()
show_similar_documents(list(dataset.keys()), cosine_similarities, sim_vecs, corpus_clans, top_sentence_indices)

search time: 451.68 ms

Top-1, Similarity = 0.44657763838768005
body: 185.html, 
Most similar articles: 
	Додатна примања странца резидента, запосленог код резидентног лица или у сталној пословној јединици нерезидентног лица, која су исплаћена до дана ступања на снагу овог закона, ослобођена су од пореза на зараду у складу са прописима који су важили до дана ступања на снагу овог закона. Примања физичког лица – странца резидента из става 1. овог члана, која су исплаћена после ступања на снагу овог закона, рачунаће се у доходак из члана 87. став 7. Закона о порезу на доходак грађана ("Службени гласник РС”, бр. 24/01, 80/02, 80/02 – др. закон, 135/04, 62/06 и 65/06 – исправка) за утврђивање годишњег пореза на доходак грађана.
	Брисан је (види члан 60. Закона - 47/2013-18) Доспелост пореза
	Стопа пореза на приходе од самосталне делатности износи 10%. *Службени гласник РС, број 135/2004 Порески подстицаји

Top-2, Similarity = 0.4285552501678467
body: 421.html, 
Most similar articles: 
	(1)

<ipython-input-81-383e26706464>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  top_sentence_indices = np.array(top_sentence_indices)[most_similar_doc_indices]


In [97]:
query = '''
 ЗАКОН

о давању контрагаранције Републике Србије Србији и Црној Гори по кредиту Међународног удружења за развој (Кредит за развој приватног и финансијског сектора)

„Службени гласник РС”, број 34 од 18. априла 2006.

Члан 1.

Република Србија преузима обавезу да као контрагарант измири обавезе Србије и Црне Горе, у износу од 68.100.000,00 специјалних права вучења, по основу Споразума о кредиту за развој – Кредит за развој приватног и финансијског сектора између Савезне Републике Југославије и Међународног удружења за развој, потписаног 4. јуна 2002. године, који је потврђен Законом о потврђивању Споразума о кредиту за развој – Кредит за развој приватног и финансијског сектора између Савезне Републике Југославије и Међународног удружења за развој („Службени лист СРЈ – Међународни уговори”, број 6/02).

Члан 2.

Контрагаранција из члана 1. овог закона даје се Србији и Црној Гори, на име обавезе из Закона о задужењу Савезне Републике Југославије код Међународног удружења за развој („Службени лист СРЈ”, број 36/02).

Члан 3.

Корисник кредита по основу Споразума о кредиту за развој из члана 1. овог закона је Република Србија (у даљем тексту: Корисник кредита).

Члан 4.

Средства за отплату кредита по основу Споразума о кредиту за развој из члана 1. овог закона, обезбедиће Корисник кредита.

Корисник кредита је дужан да средства за отплату кредита обезбеђује према плану отплате за сва повучена средства, у износу који укључује доспелу главницу, трошкове на одобрена средства, манипулативне трошкове кредита и остале трошкове.

Корисник кредита ће обезбеђивати средства за отплату кредита у буџету Републике Србије и вршити уплату доспелих обавеза на посебан рачун Србије и Црне Горе код Народне банке Србије, са искључивом наменом отплате кредита.

Члан 5.

Овај закон ступа на снагу осмог дана од дана објављивања у „Службеном гласнику Републике Србије”.
'''

In [54]:
list(dataset.items())[0][1][1]

'потврђује се споразум владе републике србије владе републике молдавије сарадњи области одбране је потписан београду децембра године оригиналу српском молдавском енглеском језику'

In [1]:
query_vec = query_to_vec("потврђује се споразум владе републике србије владе републике молдавије области одбране је потписан године оригиналу српском молдавском енглеском језику", model)
query_vec = query_vec.reshape(1,-1)
query_vec2 = query_to_vec(list(dataset.items())[0][1][1], model)
query_vec2 = query_vec2.reshape(1,-1)
cosine_similarity(query_vec, query_vec2)

NameError: name 'query_to_vec' is not defined

In [92]:
for file_path in os.listdir(dataset_path):
    print(file_path)

769.html
142.html
1257.html
193.html
593.html
126.html
153.html
309.html
185.html
383.html
833.html
777.html
623.html
626.html
799.html
342.html
1216.html
713.html
1207.html
128.html
143.html
382.html
838.html
968.html
998.html
1208.html
771.html
105.html
580.html
1198.html
680.html
647.html
936.html
401.html
773.html
175.html
688.html
230.html
1173.html
48.html
440.html
124.html
960.html
399.html
1265.html
135.html
747.html
351.html
880.html
611.html
562.html
868.html
1012.html
568.html
1078.html
301.html
1022.html
1299.html
615.html
561.html
677.html
1250.html
1310.html
80.html
83.html
29.html
295.html
520.html
913.html
58.html
68.html
411.html
985.html
460.html
563.html
435.html
1210.html
770.html
181.html
321.html
1175.html
1117.html
655.html
102.html
692.html
693.html
106.html
557.html
775.html
1032.html
714.html
1028.html
260.html
178.html
701.html
212.html
779.html
415.html
112.html
1001.html
88.html
1196.html
832.html
524.html
436.html
339.html
598.html
426.html
1249.html
231.h